# Regress out mutation load from functional mutation enrichment

Script related to figure 4d, s4k and s4l

### Aim:
- Regress-out mutational load per sample from mutation enrichment score per sample

In [1]:
sessionInfo()

R version 3.5.0 (2018-04-23)
Platform: x86_64-apple-darwin15.6.0 (64-bit)
Running under: macOS High Sierra 10.13.6

Matrix products: default
BLAS: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRblas.0.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRlapack.dylib

locale:
[1] C/UTF-8/C/C/C/C

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] compiler_3.5.0  IRdisplay_0.6.1 pbdZMQ_0.3-3    tools_3.5.0    
 [5] htmltools_0.3.6 base64enc_0.1-3 crayon_1.3.4    Rcpp_1.0.1     
 [9] uuid_0.1-2      IRkernel_0.8.14 jsonlite_1.6    digest_0.6.18  
[13] repr_0.17       evaluate_0.13  

## Install packages

In [ ]:
#install.packages("lmtest")

## Load packages

In [2]:
library(lmtest)

Warning message:
“package ‘lmtest’ was built under R version 3.5.2”Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



In [ ]:
setwd("~/git_repos/HGSOC_TME_Heterogeneity/")

# Load data

In [10]:
mutpath <- read.csv("../../Data/4/hallmark_mutation_load.txt",
                   row.names="X")

In [11]:
mutpath <- t(mutpath)

In [12]:
head(mutpath)

,HALLMARK_TNFA_SIGNALING_VIA_NFKB,HALLMARK_HYPOXIA,HALLMARK_CHOLESTEROL_HOMEOSTASIS,HALLMARK_MITOTIC_SPINDLE,HALLMARK_WNT_BETA_CATENIN_SIGNALING,HALLMARK_TGF_BETA_SIGNALING,HALLMARK_IL6_JAK_STAT3_SIGNALING,HALLMARK_DNA_REPAIR,HALLMARK_G2M_CHECKPOINT,HALLMARK_APOPTOSIS,⋯,HALLMARK_IL2_STAT5_SIGNALING,HALLMARK_BILE_ACID_METABOLISM,HALLMARK_PEROXISOME,HALLMARK_ALLOGRAFT_REJECTION,HALLMARK_SPERMATOGENESIS,HALLMARK_KRAS_SIGNALING_UP,HALLMARK_KRAS_SIGNALING_DN,HALLMARK_PANCREAS_BETA_CELLS,StromalSignature,ImmuneSignature
RG10T17,3,2,2,4,1,0,0,1,5,0,⋯,2,1,0,4,1,1,2,0,0,2
RG10T18,3,1,2,3,1,0,0,1,3,0,⋯,2,2,0,3,1,0,1,0,1,1
RG10T19,2,1,2,4,1,0,0,1,5,0,⋯,2,1,0,3,1,1,1,0,0,1
RG10T5,2,1,2,3,1,0,0,1,4,0,⋯,2,1,0,3,1,1,3,0,0,1
RG10T7,2,1,2,4,1,0,0,1,5,0,⋯,2,1,0,3,1,1,4,0,0,1
RG10T8,2,1,2,4,1,0,0,1,4,0,⋯,2,2,1,3,1,0,1,0,0,1


In [6]:
colnames(mutpath)

[1] "HALLMARK_TNFA_SIGNALING_VIA_NFKB"          
 [2] "HALLMARK_HYPOXIA"                          
 [3] "HALLMARK_CHOLESTEROL_HOMEOSTASIS"          
 [4] "HALLMARK_MITOTIC_SPINDLE"                  
 [5] "HALLMARK_WNT_BETA_CATENIN_SIGNALING"       
 [6] "HALLMARK_TGF_BETA_SIGNALING"               
 [7] "HALLMARK_IL6_JAK_STAT3_SIGNALING"          
 [8] "HALLMARK_DNA_REPAIR"                       
 [9] "HALLMARK_G2M_CHECKPOINT"                   
[10] "HALLMARK_APOPTOSIS"                        
[11] "HALLMARK_NOTCH_SIGNALING"                  
[12] "HALLMARK_ADIPOGENESIS"                     
[13] "HALLMARK_ESTROGEN_RESPONSE_EARLY"          
[14] "HALLMARK_ESTROGEN_RESPONSE_LATE"           
[15] "HALLMARK_ANDROGEN_RESPONSE"                
[16] "HALLMARK_MYOGENESIS"                       
[17] "HALLMARK_PROTEIN_SECRETION"                
[18] "HALLMARK_INTERFERON_ALPHA_RESPONSE"        
[19] "HALLMARK_INTERFERON_GAMMA_RESPONSE"        
[20] "HALLMARK_APICAL_JUNCTION"                  
[21] "HALLMARK_APICAL_SURFACE"                   
[22] "HALLMARK_HEDGEHOG_SIGNALING"               
[23] "HALLMARK_COMPLEMENT"                       
[24] "HALLMARK_UNFOLDED_PROTEIN_RESPONSE"        
[25] "HALLMARK_PI3K_AKT_MTOR_SIGNALING"          
[26] "HALLMARK_MTORC1_SIGNALING"                 
[27] "HALLMARK_E2F_TARGETS"                      
[28] "HALLMARK_MYC_TARGETS_V1"                   
[29] "HALLMARK_MYC_TARGETS_V2"                   
[30] "HALLMARK_EPITHELIAL_MESENCHYMAL_TRANSITION"
[31] "HALLMARK_INFLAMMATORY_RESPONSE"            
[32] "HALLMARK_XENOBIOTIC_METABOLISM"            
[33] "HALLMARK_FATTY_ACID_METABOLISM"            
[34] "HALLMARK_OXIDATIVE_PHOSPHORYLATION"        
[35] "HALLMARK_GLYCOLYSIS"                       
[36] "HALLMARK_REACTIVE_OXIGEN_SPECIES_PATHWAY"  
[37] "HALLMARK_P53_PATHWAY"                      
[38] "HALLMARK_UV_RESPONSE_UP"                   
[39] "HALLMARK_UV_RESPONSE_DN"                   
[40] "HALLMARK_ANGIOGENESIS"                     
[41] "HALLMARK_HEME_METABOLISM"                  
[42] "HALLMARK_COAGULATION"                      
[43] "HALLMARK_IL2_STAT5_SIGNALING"              
[44] "HALLMARK_BILE_ACID_METABOLISM"             
[45] "HALLMARK_PEROXISOME"                       
[46] "HALLMARK_ALLOGRAFT_REJECTION"              
[47] "HALLMARK_SPERMATOGENESIS"                  
[48] "HALLMARK_KRAS_SIGNALING_UP"                
[49] "HALLMARK_KRAS_SIGNALING_DN"                
[50] "HALLMARK_PANCREAS_BETA_CELLS"              
[51] "StromalSignature"                          
[52] "ImmuneSignature"

In [7]:
purity <- read.csv("../../Data/4/ovct_wes_purity.csv",
                  row.names="X")

purity <- t(purity)

head(purity)

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '../../Data/4/ovct_wes_purity.csv'”

,ovct_wes_purity,ovct_wes_purity_val
RG10T17,High,0.6496
RG10T18,High,0.6706
RG10T19,High,0.6695
RG10T5,Low,0.5973
RG10T7,Low,0.5065
RG10T8,Low,0.4551


In [8]:
dim(purity)

[1] 50  2

In [9]:
mutload <- read.csv("../../Data/4/mutation_load_per_sample2.csv",
                    row.names="X")

mutload <- t(mutload)

head(mutload)

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '../../Data/4/mutation_load_per_sample2.csv'”

,mutation_load_per_sample
RG10T17,250
RG10T18,226
RG10T19,222
RG10T5,230
RG10T7,235
RG10T8,225


In [10]:
dim(mutload)

[1] 50  1

In [11]:
dim(mutpath)

[1] 50 52

In [13]:
patient_id <- read.csv("../../Data/4/Patient_ID.csv", row.names="X")

head(patient_id)


,patient_id
RG20T8,#8DA0CA
RG20T14,#8DA0CA
RG20T10,#8DA0CA
RG1T6,#e41a1c
RG20T12,#8DA0CA
RG17T19,#f781bf


In [14]:
str(patient_id)

'data.frame':	50 obs. of  1 variable:
 $ patient_id: Factor w/ 10 levels "#377eb8","#4daf4a",..: 4 4 4 7 4 8 5 5 7 6 ...


In [15]:
dim(mutpath)[2]

[1] 52

In [16]:
pathways <- c()
p_vals <- c()

for(i in 1:dim(mutpath)[2]) { 
    
    mod1 <- lm(purity[,"ovct_wes_purity_val"] ~ mutpath[,i]+mutload[,"mutation_load_per_sample"]+patient_id[,"patient_id"])
    
    mod2 <- lm(purity[,"ovct_wes_purity_val"] ~ mutload[,"mutation_load_per_sample"]+patient_id[,"patient_id"])
    
    pathways <- append(pathways, colnames(mutpath)[i])
    p_vals <- append(p_vals, lrtest(mod1, mod2)[2,"Pr(>Chisq)"])
    
    if (lrtest(mod1, mod2)[2,"Pr(>Chisq)"] < 0.05){
    
        print(colnames(mutpath)[i])
        print(lrtest(mod1, mod2)[2,"Pr(>Chisq)"])}
}

[1] "HALLMARK_CHOLESTEROL_HOMEOSTASIS"
[1] 0.04871907
[1] "HALLMARK_MITOTIC_SPINDLE"
[1] 0.01968923
[1] "HALLMARK_WNT_BETA_CATENIN_SIGNALING"
[1] 0.0007433433
[1] "HALLMARK_APOPTOSIS"
[1] 1.644844e-05
[1] "HALLMARK_NOTCH_SIGNALING"
[1] 0.00787585
[1] "HALLMARK_APICAL_JUNCTION"
[1] 0.01145181
[1] "HALLMARK_HEDGEHOG_SIGNALING"
[1] 0.0314063
[1] "HALLMARK_COMPLEMENT"
[1] 0.008036277
[1] "HALLMARK_MTORC1_SIGNALING"
[1] 0.01116351
[1] "HALLMARK_XENOBIOTIC_METABOLISM"
[1] 0.02482434
[1] "HALLMARK_REACTIVE_OXIGEN_SPECIES_PATHWAY"
[1] 1.681517e-05
[1] "HALLMARK_HEME_METABOLISM"
[1] 0.006385252
[1] "HALLMARK_BILE_ACID_METABOLISM"
[1] 0.006732046
[1] "StromalSignature"
[1] 0.0001783432


In [17]:
length(pathways)

[1] 52

In [18]:
fdrs <- p.adjust(p_vals, method = "fdr", n = length(p_vals))

In [19]:
for(i in 1:length(fdrs)) { 
    
    fdr <- fdrs[i]
    
    if (fdr < 0.01){
    
        print(pathways[i])
        print(fdr)}
}

[1] "HALLMARK_WNT_BETA_CATENIN_SIGNALING"
[1] 0.009663463
[1] "HALLMARK_APOPTOSIS"
[1] 0.0004371944
[1] "HALLMARK_REACTIVE_OXIGEN_SPECIES_PATHWAY"
[1] 0.0004371944
[1] "StromalSignature"
[1] 0.003091282


In [20]:
fdrs

[1] 0.5064720255 0.2833547797 0.1809565455 0.0930763493 0.0096634631
 [6] 0.6278073988 0.5667304746 0.2833547797 0.3176547670 0.0004371944
[11] 0.0522357995 0.9926886232 0.8127260413 0.5064720255 0.4873801917
[16] 0.7529070414 0.5667304746 0.8893768138 0.8903852870 0.0595494195
[21] 0.9219612199 0.1256251892 0.0522357995 0.8541120604 0.5135590237
[26] 0.0595494195 0.9613848053 0.5899551410 0.9219612199 0.9737183362
[31] 0.5667304746 0.1075721208 0.9219612199 0.5064720255 0.5667304746
[36] 0.0004371944 0.2342871381 0.9219612199 0.5064720255 0.8541120604
[41] 0.0522357995 0.9219612199 0.9219612199 0.0522357995 0.9219612199
[46] 0.4873801917 0.7615622736 0.5064720255 0.9926886232 0.9219612199
[51] 0.0030912824 0.8541120604

In [21]:
fdrs_df <- data.frame("hallmarks" = pathways , "fdr" = fdrs)

In [22]:
tail(fdrs_df)

,hallmarks,fdr
47,HALLMARK_SPERMATOGENESIS,0.761562274
48,HALLMARK_KRAS_SIGNALING_UP,0.506472026
49,HALLMARK_KRAS_SIGNALING_DN,0.992688623
50,HALLMARK_PANCREAS_BETA_CELLS,0.921961220
51,StromalSignature,0.003091282
52,ImmuneSignature,0.854112060


In [23]:
dim(fdrs_df)

[1] 52  2

In [24]:
write.table(fdrs_df, "../../Data/4/fdrs_df.csv", sep=",", row.names = FALSE)

# End script